In [ ]:
import scanpy as sc
random_seed_configs = [(42, 500), (42, 1000), (42, 1500), (42, 2000)]
dim_size_configs = [(214, 2000), (3490, 2000), (8480, 2000), (11552, 2000), (14590, 2000),
                    (15321, 2000), (22688, 2000), (23419, 2000), (24298, 2000), (24598, 2000)]

def to_dataframe(res):
    df_data = []
    for metric, metric_res in method_res.items():
        metric = short_metric_name(metric)
        print(metric)
        for group, values in metric_res.items():
            group = dis_group_name(group, metric)
            print("\t", group, ": ", np.mean(values))
            for a_score in values:
                df_data.append({"method":method,
                                "metric":metric,
                                "group": group,
                                "score": a_score
                                })
    df_data = pd.DataFrame(df_data)
    return df_data

settings = {"dentategyrus":{
                'CBDir_input':[('OPC', "OL")],
                'ICVCoh_input':['OPC', "OL"],
                'k_cluster': "clusters",
                'prefix': "ld_dentategyrus"
            },
            "scEU":{
                'CBDir_input':[],,
                'ICVCoh_input':["1", "2"]
                'k_cluster': "monocle_branch_id",
                'prefix': "ld_scEU"
            },
            "scNT":{
                'CBDir_input':[("0", "15"), ("15", "30"), ("30", "60"), ("60", "120")],
                'ICVCoh_input':["0", "15", "30", "60", "120"],
                'k_cluster': "time",
                'prefix': "ld_scNT"
            },
            "pancreas":{
                'CBDir_input':[("Ngn3 low EP", "Ngn3 high EP"),
                            ("Ngn3 high EP", "Fev+"),
                            ("Fev+", "Alpha"),
                            ("Fev+", "Beta"),
                            ("Fev+", "Delta"),
                            ("Fev+", "Epsilon")],
                'ICVCoh_input':["Ngn3 low EP", "Ngn3 high EP", "Fev+", "Alpha", "Beta", "Delta", "Epsilon"],
                'k_cluster': "clusters",
                'prefix': "ld_pancreas"
            },
            "Erythroid_mouse":{
                'CBDir_input':[("Erythroid1", "Erythroid2"), ('Erythroid2', "Erythroid3")],
                'ICVCoh_input':["Erythroid1", "Erythroid2", "Erythroid3"],
                'k_cluster': "celltype",
                'prefix': "ld_Ery_mouse"
            },
            "Erythroid_human":{
                'CBDir_input':[("Early Erythroid", "Mid  Erythroid"), ('Mid  Erythroid', "Late Erythroid")],
                'ICVCoh_input':["Early Erythroid", "Mid  Erythroid", "Late Erythroid"],
                'k_cluster': "type2",
                'prefix': "ld_Ery_human"
            },
}}

def short_metric_name(metric):
    return {"Cross-Boundary Transition Score (A->B)":"CBTrans",
                  "Cross-Boundary Velocity Coherence (A->B)":"CBVCoh",
                  "Cross-Boundary Direction Correctness (A->B)":"CBDir",
                  "In-cluster Confidence":"ICConf",
                  "In-cluster Coherence":"ICVCoh"
                 }[metric]

def score(CBDir_input, ICVCoh_input, k_cluster, file_path):
    adata = sc.read_h5ad(file_path)
    if file_path.startswith("ld_scNT"):
        adata.obs['time'] = adata.obs.time.astype('category')
    scv.tl.velocity_confidence(adata, vkey='velocity')
    scores = evaluate(adata, CBDir_input, k_cluster, "velocity")
    CBDir_all_scores = scores['Cross-Boundary Direction Correctness (A->B)']
    ICVCoh_all_scores = scores['In-cluster Coherence']
    CBDir_score_by_group, _ = summary_scores(CBDir_all_scores)
    CBDir_score = np.mean([CBDir_score_by_group[g] if g in CBDir_score_by_group else 0 for g in CBDir_input])
    ICVCoh_score_by_group, _ = summary_scores(ICVCoh_all_scores)
    ICVCoh_score = np.mean([ICVCoh_score_by_group[g] if g in ICVCoh_score_by_group else 0 for g in ICVCoh_input])
    return CBDir_score, ICVCoh_score



    
